# Yandex.Music

## Introduction <a id='intro'></a>
Whenever we're doing research, we need to formulate hypotheses that we can then test. Sometimes we accept these hypotheses; other times, we reject them. To make the right decisions, a business must be able to understand whether or not it's making the right assumptions.

In this project, you'll compare the music preferences of the cities of Springfield and Shelbyville. You'll study real Yandex.Music data to test the hypotheses below and compare user behavior for these two cities.

### Goal: 
Test three hypotheses:
1. User activity differs depending on the day of the week and from city to city. 
2. On Monday mornings, Springfield and Shelbyville residents listen to different genres. This is also true for Friday evenings. 
3. Springfield and Shelbyville listeners have different preferences. In Springfield, they prefer pop, while Shelbyville has more rap fans.

### Stages 
Data on user behavior is stored in the file `/datasets/music_project_en.csv`. There is no information about the quality of the data, so you will need to explore it before testing the hypotheses. 

First, you'll evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, you will try to account for the most critical problems.
 
Your project will consist of three stages:
 1. Data overview
 2. Data preprocessing
 3. Testing the hypotheses
 
[Back to Contents](#back)

## Stage 1. Data overview <a id='data_review'></a>

Open the data on Yandex.Music and explore it.

You'll need `pandas`, so import it.

In [1]:
import pandas as pd

Read the file `music_project_en.csv` from the `/datasets/` folder and save it in the `df` variable:

In [2]:
file_path = '/datasets/music_project_en.csv'
df = pd.read_csv(file_path)
description = df.describe()
# reading the file and storing it to df 


Print the first 10 table rows:

In [3]:
display(description) # obtaining the first 10 rows from the df table

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


Obtaining the general information about the table with one command:

In [4]:
info_summary = df.info()    # obtaining general information about the data in df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


The table contains seven columns. They all store the same data type: `object`.

According to the documentation:
- `'userID'` — user identifier
- `'Track'` — track title
- `'artist'` — artist's name
- `'genre'`
- `'City'` — user's city
- `'time'` — the exact time the track was played
- `'Day'` — day of the week

We can see three issues with style in the column names:
1. Some names are uppercase, some are lowercase.
2. There are spaces in some names.
3. `Detect the third issue yourself and describe it here`.

The number of column values is different. This means the data contains missing values.


### Conclusions <a id='data_review_conclusions'></a> 

Each row in the table stores data on a track that was played. Some columns describe the track itself: its title, artist and genre. The rest convey information about the user: the city they come from, the time they played the track. 

It's clear that the data is sufficient to test the hypotheses. However, there are missing values.

To move forward, we need to preprocess the data.

## Stage 2. Data preprocessing <a id='data_preprocessing'></a>
Correct the formatting in the column headers and deal with the missing values. Then, check whether there are duplicates in the data.

### Header style <a id='header_style'></a>
Print the column header:

In [5]:
column_names = df.columns
for column in column_names:
    print(column) # the list of column names in the df table

  userID
Track
artist
genre
  City  
time
Day


Change column names according to the rules of good style:
* If the name has several words, use snake_case
* All characters must be lowercase
* Delete spaces

In [6]:
df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
df.columns = df.columns.str.lower()  # Convert to lowercase
df.columns = df.columns.str.replace(' ', '_', regex=False)  # renaming columns

<div class="alert alert-block alert-success">✔️
    

__Reviewer's comment №1__

Well done

Check the result. Print the names of the columns once more:

In [7]:
for column in df.columns:
    print(column) # checking result: the list of column names

userid
track
artist
genre
city
time
day


### Missing values <a id='missing_values'></a>
First, find the number of missing values in the table. To do so, use two `pandas` methods:

In [8]:
missing_values_count = df.isnull().sum() # calculating missing values

Not all missing values affect the research. For instance, the missing values in `track` and `artist` are not critical. You can simply replace them with clear markers.

But missing values in `'genre'` can affect the comparison of music preferences in Springfield and Shelbyville. In real life, it would be useful to learn the reasons why the data is missing and try to make up for them. But we do not have that opportunity in this project. So you will have to:
* Fill in these missing values with markers
* Evaluate how much the missing values may affect your computations

Replace the missing values in `'track'`, `'artist'`, and `'genre'` with the string `'unknown'`. To do this, create the `columns_to_replace` list, loop over it with `for`, and replace the missing values in each of the columns:

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column].fillna('unknown', inplace=True) # looping over column names and replacing missing values with 'unknown'

Make sure the table contains no more missing values. Count the missing values again.

In [10]:
df.isnull().sum()  # counting missing values

userid    0
track     0
artist    0
genre     0
city      0
time      0
day       0
dtype: int64

### Duplicates <a id='duplicates'></a>
Find the number of obvious duplicates in the table using one command:

In [11]:
num_duplicates = df.duplicated().sum()
 # counting clear duplicates

Call the `pandas` method for getting rid of obvious duplicates:

In [12]:
df= df.drop_duplicates()
 # removing obvious duplicates

Count obvious duplicates once more to make sure you have removed all of them:

In [13]:
df_no_duplicates = df.drop_duplicates()
num_duplicates_after_removal = df_no_duplicates.duplicated().sum()
print("Number of obvious duplicates after removal:", num_duplicates_after_removal)


Number of obvious duplicates after removal: 0


Now get rid of implicit duplicates in the `genre` column. For example, the name of a genre can be written in different ways. Such errors will also affect the result.

Print a list of unique genre names, sorted in alphabetical order. To do so:
* Retrieve the intended DataFrame column 
* Apply a sorting method to it
* For the sorted column, call the method that will return all unique column values

In [14]:
sorted_unique_genres = df['genre'].sort_values().unique()
for genre in sorted_unique_genres:
    print(genre) # viewing unique genre names

acid
acoustic
action
adult
africa
afrikaans
alternative
ambient
americana
animated
anime
arabesk
arabic
arena
argentinetango
art
audiobook
avantgarde
axé
baile
balkan
beats
bigroom
black
bluegrass
blues
bollywood
bossa
brazilian
breakbeat
breaks
broadway
cantautori
cantopop
canzone
caribbean
caucasian
celtic
chamber
children
chill
chinese
choral
christian
christmas
classical
classicmetal
club
colombian
comedy
conjazz
contemporary
country
cuban
dance
dancehall
dancepop
dark
death
deep
deutschrock
deutschspr
dirty
disco
dnb
documentary
downbeat
downtempo
drum
dub
dubstep
eastern
easy
electronic
electropop
emo
entehno
epicmetal
estrada
ethnic
eurofolk
european
experimental
extrememetal
fado
film
fitness
flamenco
folk
folklore
folkmetal
folkrock
folktronica
forró
frankreich
französisch
french
funk
future
gangsta
garage
german
ghazal
gitarre
glitch
gospel
gothic
grime
grunge
gypsy
handsup
hard'n'heavy
hardcore
hardstyle
hardtechno
hip
hip-hop
hiphop
historisch
holiday
hop
horror
house
idm
i

Look through the list to find implicit duplicates of the genre `hiphop`. These could be names written incorrectly or alternative names of the same genre.

You will see the following implicit duplicates:
* `hip`
* `hop`
* `hip-hop`

To get rid of them, declare the function `replace_wrong_genres()` with two parameters: 
* `wrong_genres=` — the list of duplicates
* `correct_genre=` — the string with the correct value

The function should correct the names in the `'genre'` column from the `df` table, i.e. replace each value from the `wrong_genres` list with the value in `correct_genre`.

In [15]:
def replace_wrong_genres (wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre) 
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hip-hop'
replace_wrong_genres(wrong_genres, correct_genre)


Call `replace_wrong_genres()` and pass it arguments so that it clears implicit duplcates (`hip`, `hop`, and `hip-hop`) and replaces them with `hiphop`:

In [16]:
def replace_wrong_genres (wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre) 
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(wrong_genres, correct_genre)
 # removing implicit duplicates

Make sure the duplicate names were removed. Print the list of unique values from the `'genre'` column:

In [17]:
unique_genres = df['genre'].unique()
print(unique_genres) # checking for implicit duplicates

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hiphop' 'jazz' 'postrock'
 'latin' 'classical' 'metal' 'reggae' 'triphop' 'blues' 'instrumental'
 'rusrock' 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'salsa' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'tropical' 'contemporary' 'new' 'soul' 'holiday'
 'german' 'jpop' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'korean'
 'numetal' 'vocal' 'estrada' 'tango' 'loungeelectronic' 'classicmetal'
 'dubstep' 'club' 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french'
 'disco' 'religious' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'swing' 'breaks' 'eurofolk'
 'stone

### Conclusions <a id='data_preprocessing_conclusions'></a>
We detected three issues with the data:

- Incorrect header styles
- Missing values
- Obvious and implicit duplicates

The headers have been cleaned up to make processing the table simpler.

All missing values have been replaced with `'unknown'`. But we still have to see whether the missing values in `'genre'` will affect our calculations.

The absence of duplicates will make the results more precise and easier to understand.

Now we can move on to testing hypotheses. 

## Stage 3. Testing hypotheses <a id='hypotheses'></a>

### Hypothesis 1: comparing user behavior in two cities <a id='activity'></a>

According to the first hypothesis, users from Springfield and Shelbyville listen to music differently. Test this using the data on three days of the week: Monday, Wednesday, and Friday.

* Divide the users into groups by city.
* Compare how many tracks each group played on Monday, Wednesday, and Friday.


For the sake of practice, perform each computation separately. 

Evaluate user activity in each city. Group the data by city and find the number of songs played in each group.



In [18]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_') 
grouped_data = df.groupby(['city', 'day'])['track'].count()
print(grouped_data)
 # Counting up the tracks played in each city

city         day      
Shelbyville  Friday        5895
             Monday        5614
             Wednesday     7003
Springfield  Friday       15945
             Monday       15740
             Wednesday    11056
Name: track, dtype: int64


Springfield has more tracks played than Shelbyville. But that does not imply that citizens of Springfield listen to music more often. This city is simply bigger, and there are more users.

Now group the data by day of the week and find the number of tracks played on Monday, Wednesday, and Friday.


In [19]:
grouped_data = df.groupby('day')['track'].count()
print(grouped_data)  # Calculating tracks played on each of the three days

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


Wednesday is the quietest day overall. But if we consider the two cities separately, we might come to a different conclusion.

You have seen how grouping by city or day works. Now write a function that will group by both.

Create the `number_tracks()` function to calculate the number of songs played for a given day and city. It will require two parameters:
* day of the week
* name of the city

In the function, use a variable to store the rows from the original table, where:
  * `'day'` column value is equal to the `day` parameter
  * `'city'` column value is equal to the `city` parameter

Apply consecutive filtering with logical indexing.

Then calculate the `'user_id'` column values in the resulting table. Store the result to a new variable. Return this variable from the function.

In [20]:
import pandas as pd


file_path = '/datasets/music_project_en.csv'
df = pd.read_csv(file_path)


df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
display(df)

def number_tracks(day, city):
    filtered_data = df.loc[(df['day'] == day) & (df['city'] == city)]
    display(filtered_data)
    tracks_played = filtered_data['userid'].count()  # Count the number of 'user_id' values
    return tracks_played



# We'll declare a function with two parameters: day=, city=.
# Let the track_list variable store the df rows where
# the value in the 'day' column is equal to the day= parameter and, at the same time, 
# the value in the 'city' column is equal to the city= parameter (apply consecutive filtering 
# with logical indexing).
# Let the track_list_count variable store the number of 'user_id' column values in track_list
# (found with the count() method).
# Let the function return a number: the value of track_list_count.

# The function counts tracked played for a certain city and day.
# It first retrieves the rows with the intended day from the table,
# then filters out the rows with the intended city from the result,
# then finds the number of 'user_id' values in the filtered table,
# then returns that number.
# To see what it returns, wrap the function call in print().

,userid,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
...,...,...,...,...,...,...,...
65074,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
65075,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hip,Shelbyville,10:00:00,Monday
65076,C5E3A0D5,Jalopiina,NaN,industrial,Springfield,20:09:26,Friday
65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


Call `number_tracks()` six times, changing the parameter values, so that you retrieve the data on both cities for each of the three days.

In [21]:
springfield_on_monday = number_tracks('Monday', 'Springfield')
display(springfield_on_monday)


,userid,track,artist,genre,city,time,day
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
10,BC5A3A29,Gool la Mita,Shireen Abdul Wahab,world,Springfield,14:08:42,Monday
11,8B5192C0,Is There Anybody Out There? (Panoramic Paralys...,Pink Floyd Floydhead,electronic,Springfield,13:47:49,Monday
12,FF3FD2BD,Truth,Bamboo,pop,Springfield,09:19:49,Monday
15,E3C5756F,NaN,NaN,NaN,Springfield,09:24:51,Monday
...,...,...,...,...,...,...,...
65042,83E9C8C4,Buddhist Beat,Asian Zen Spa Music Meditation,ambient,Springfield,13:25:29,Monday
65046,9656377C,Algun Dia,Om Ovnimoon,downtempo,Springfield,08:10:59,Monday
65047,85A7CFD3,Working Class Hero,Marianne Faithfull,rock,Springfield,14:51:37,Monday
65069,BE1AAD74,Waterwalk,Eduardo Gonzales,electronic,Springfield,20:38:59,Monday


16715

In [22]:
shelbyville_on_monday = number_tracks('Monday', 'Shelbyville')


,userid,track,artist,genre,city,time,day
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
16,81D05C7D,SLAVES OF FEAR,HEALTH,alternative,Shelbyville,20:54:48,Monday
17,39DE290E,Hallo Hallo,Die Klima Hawaiians,folk,Shelbyville,14:36:47,Monday
18,58AE138A,Pat-a-Cake,Mother Goose Club,children,Shelbyville,20:04:56,Monday
26,982219FD,We Not Speak Americano,Genio Band,pop,Shelbyville,20:03:03,Monday
...,...,...,...,...,...,...,...
65064,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
65065,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
65066,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday
65067,F1B93F29,Poison Kiss,Centerstone,rock,Shelbyville,22:00:29,Monday


In [23]:
springfield_on_wednesday = number_tracks('Wednesday', 'Springfield') # the number of songs played in Springfield on Wednesday

,userid,track,artist,genre,city,time,day
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
19,772F5B59,Sweetback,The Fabulous Three,rnb,Springfield,13:56:42,Wednesday
28,D8D698A9,Romantici,Viola Valentino,pop,Springfield,08:23:21,Wednesday
36,FCDF1C8E,Nasebanaru,Zen,reggae,Springfield,20:19:36,Wednesday
...,...,...,...,...,...,...,...
65054,6A71CB92,Remind Me To Forget You,Sarah Geronimo,pop,Springfield,14:21:28,Wednesday
65057,8624CE71,Treasures and Trolls,Tera Melos,alternative,Springfield,13:02:57,Wednesday
65060,574EBCB0,Turn up the Radio,Autograph,rock,Springfield,13:45:39,Wednesday
65068,9E65547F,Fascinating Rhythm,Mark Murphy,jazz,Springfield,20:29:09,Wednesday


In [24]:
shelbyville_on_wednesday = number_tracks('Wednesday', 'Shelbyville') # the number of songs played in Shelbyville on Wednesday

,userid,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday
14,94EB25C2,Make Love Whenever You Can,Arabesque,pop,Shelbyville,13:22:08,Wednesday
22,B8E29C9C,The Drunken Scotsman,Ménage à Moi,folk,Shelbyville,20:20:49,Wednesday
...,...,...,...,...,...,...,...
65029,27047C8,Rainy Day,TumaniYO,rusrap,Shelbyville,21:52:57,Wednesday
65037,64E06740,I Used To,Lunatic Wolf,rock,Shelbyville,14:18:41,Wednesday
65040,77979A66,Money,Deep Red Wood,rusrap,Shelbyville,21:16:18,Wednesday
65045,95104ED1,Break the Mold,Authority Zero,punk,Shelbyville,13:49:40,Wednesday


In [25]:
springfield_on_friday = number_tracks('Friday', 'Springfield') # the number of songs played in Springfield on Friday

,userid,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
13,CC782B0F,After School Special,Detroit Grand Pubahs,dance,Springfield,20:04:12,Friday
34,B416A40F,Maybe,Maria Michelle,ruspop,Springfield,14:49:21,Friday
42,F9AD83A7,Fancy,Grey Killer,triphop,Springfield,14:53:39,Friday
...,...,...,...,...,...,...,...
65071,92378E24,Swing it Like You Mean it,OJOJOJ,techno,Springfield,21:12:56,Friday
65072,C532021D,We Can Not Be Silenced,Pänzer,extrememetal,Springfield,08:38:24,Friday
65076,C5E3A0D5,Jalopiina,NaN,industrial,Springfield,20:09:26,Friday
65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [26]:
shelbyville_on_friday = number_tracks('Friday', 'Shelbyville') # the number of songs played in Shelbyville on Friday

,userid,track,artist,genre,city,time,day
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
30,EBA5A8B2,Päivän Nousu Nostajani,Värttinä,world,Shelbyville,08:00:31,Friday
31,F6A44469,Veritas Universalis,Devathorn,metal,Shelbyville,20:19:42,Friday
38,14FBD1E5,Solteras,Beauty Brain,dance,Shelbyville,20:33:11,Friday
49,2A35001B,Can You Feel the Love Tonight,Disney Peaceful Piano,instrumental,Shelbyville,21:51:37,Friday
...,...,...,...,...,...,...,...
65030,E82DA566,I Hate You More Than My Life,Solar Fake,electronic,Shelbyville,20:55:55,Friday
65036,A84E445,Oasis,Dmitry Aeolus,electronic,Shelbyville,08:32:46,Friday
65038,2AD78C04,Welcome to Zombietown,Dirty Wee Middens,punk,Shelbyville,08:57:42,Friday
65043,CDCAFD62,Payday,King Hot,hip,Shelbyville,14:31:37,Friday


Use `pd.DataFrame` to create a table, where
* Column names are: `['city', 'monday', 'wednesday', 'friday']`
* The data is the results you got from `number_tracks()`

In [27]:
results = []

for city in ['Springfield', 'Shelbyville']:
    monday = number_tracks('Monday', city)
    wednesday = number_tracks('Wednesday', city)
    friday = number_tracks('Friday', city)
    
    results.append({'city': city, 'monday': monday, 'wednesday': wednesday, 'friday': friday})

results_df = pd.DataFrame(results)
print(results_df)
# table with results

,userid,track,artist,genre,city,time,day
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
10,BC5A3A29,Gool la Mita,Shireen Abdul Wahab,world,Springfield,14:08:42,Monday
11,8B5192C0,Is There Anybody Out There? (Panoramic Paralys...,Pink Floyd Floydhead,electronic,Springfield,13:47:49,Monday
12,FF3FD2BD,Truth,Bamboo,pop,Springfield,09:19:49,Monday
15,E3C5756F,NaN,NaN,NaN,Springfield,09:24:51,Monday
...,...,...,...,...,...,...,...
65042,83E9C8C4,Buddhist Beat,Asian Zen Spa Music Meditation,ambient,Springfield,13:25:29,Monday
65046,9656377C,Algun Dia,Om Ovnimoon,downtempo,Springfield,08:10:59,Monday
65047,85A7CFD3,Working Class Hero,Marianne Faithfull,rock,Springfield,14:51:37,Monday
65069,BE1AAD74,Waterwalk,Eduardo Gonzales,electronic,Springfield,20:38:59,Monday


,userid,track,artist,genre,city,time,day
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
19,772F5B59,Sweetback,The Fabulous Three,rnb,Springfield,13:56:42,Wednesday
28,D8D698A9,Romantici,Viola Valentino,pop,Springfield,08:23:21,Wednesday
36,FCDF1C8E,Nasebanaru,Zen,reggae,Springfield,20:19:36,Wednesday
...,...,...,...,...,...,...,...
65054,6A71CB92,Remind Me To Forget You,Sarah Geronimo,pop,Springfield,14:21:28,Wednesday
65057,8624CE71,Treasures and Trolls,Tera Melos,alternative,Springfield,13:02:57,Wednesday
65060,574EBCB0,Turn up the Radio,Autograph,rock,Springfield,13:45:39,Wednesday
65068,9E65547F,Fascinating Rhythm,Mark Murphy,jazz,Springfield,20:29:09,Wednesday


,userid,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
13,CC782B0F,After School Special,Detroit Grand Pubahs,dance,Springfield,20:04:12,Friday
34,B416A40F,Maybe,Maria Michelle,ruspop,Springfield,14:49:21,Friday
42,F9AD83A7,Fancy,Grey Killer,triphop,Springfield,14:53:39,Friday
...,...,...,...,...,...,...,...
65071,92378E24,Swing it Like You Mean it,OJOJOJ,techno,Springfield,21:12:56,Friday
65072,C532021D,We Can Not Be Silenced,Pänzer,extrememetal,Springfield,08:38:24,Friday
65076,C5E3A0D5,Jalopiina,NaN,industrial,Springfield,20:09:26,Friday
65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


,userid,track,artist,genre,city,time,day
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
16,81D05C7D,SLAVES OF FEAR,HEALTH,alternative,Shelbyville,20:54:48,Monday
17,39DE290E,Hallo Hallo,Die Klima Hawaiians,folk,Shelbyville,14:36:47,Monday
18,58AE138A,Pat-a-Cake,Mother Goose Club,children,Shelbyville,20:04:56,Monday
26,982219FD,We Not Speak Americano,Genio Band,pop,Shelbyville,20:03:03,Monday
...,...,...,...,...,...,...,...
65064,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
65065,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
65066,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday
65067,F1B93F29,Poison Kiss,Centerstone,rock,Shelbyville,22:00:29,Monday


,userid,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday
14,94EB25C2,Make Love Whenever You Can,Arabesque,pop,Shelbyville,13:22:08,Wednesday
22,B8E29C9C,The Drunken Scotsman,Ménage à Moi,folk,Shelbyville,20:20:49,Wednesday
...,...,...,...,...,...,...,...
65029,27047C8,Rainy Day,TumaniYO,rusrap,Shelbyville,21:52:57,Wednesday
65037,64E06740,I Used To,Lunatic Wolf,rock,Shelbyville,14:18:41,Wednesday
65040,77979A66,Money,Deep Red Wood,rusrap,Shelbyville,21:16:18,Wednesday
65045,95104ED1,Break the Mold,Authority Zero,punk,Shelbyville,13:49:40,Wednesday


,userid,track,artist,genre,city,time,day
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
30,EBA5A8B2,Päivän Nousu Nostajani,Värttinä,world,Shelbyville,08:00:31,Friday
31,F6A44469,Veritas Universalis,Devathorn,metal,Shelbyville,20:19:42,Friday
38,14FBD1E5,Solteras,Beauty Brain,dance,Shelbyville,20:33:11,Friday
49,2A35001B,Can You Feel the Love Tonight,Disney Peaceful Piano,instrumental,Shelbyville,21:51:37,Friday
...,...,...,...,...,...,...,...
65030,E82DA566,I Hate You More Than My Life,Solar Fake,electronic,Shelbyville,20:55:55,Friday
65036,A84E445,Oasis,Dmitry Aeolus,electronic,Shelbyville,08:32:46,Friday
65038,2AD78C04,Welcome to Zombietown,Dirty Wee Middens,punk,Shelbyville,08:57:42,Friday
65043,CDCAFD62,Payday,King Hot,hip,Shelbyville,14:31:37,Friday


          city  monday  wednesday  friday
0  Springfield   16715      11755   16890
1  Shelbyville    5982       7478    6259


**Conclusions**

The data reveals differences in user behavior:

- In Springfield, the number of songs played peaks on Mondays and Fridays, while on Wednesday there is a decrease in activity.
- In Shelbyville, on the contrary, users listen to music more on Wednesday. User activity on Monday and Friday is smaller.

So the first hypothesis seems to be correct.

### Hypothesis 2: music at the beginning and end of the week <a id='week'></a>

According to the second hypothesis, on Monday morning and Friday night, citizens of Springfield listen to genres that differ from ones users from Shelbyville enjoy.

Get tables (make sure that the name of your combined table matches the DataFrame given in the two code blocks below):
* For Springfield — `spr_general`
* For Shelbyville — `shel_general`

In [28]:
def filter_data(city, day_part):
    filtered_data = df[(df['city'] == city) & (df['time'] == day_part)]
    return filtered_data
monday_morning = '06:00-11:59'
friday_night = '18:00-23:59'

spr_monday_morning = filter_data('Springfield', monday_morning)
spr_friday_night = filter_data('Springfield', friday_night)

In [29]:
def filter_data(city, day_part):
    filtered_data = df[(df['city'] == city) & (df['time'] == day_part)]
    return filtered_data
monday_morning = '06:00-11:59'
friday_night = '18:00-23:59'

shel_monday_morning = filter_data('Shelbyville', monday_morning)
shel_friday_night = filter_data('Shelbyville', friday_night)

spr_general = pd.concat([spr_monday_morning, spr_friday_night])
shel_general = pd.concat([shel_monday_morning, shel_friday_night])


Write the `genre_weekday()` function with four parameters:
* A table for data (`df`)
* The day of the week (`day`)
* The first timestamp, in 'hh:mm' format (`time1`)
* The last timestamp, in 'hh:mm' format (`time2`)

The function should return info on the 15 most popular genres on a given day within the period between the two timestamps.

In [30]:
import pandas as pd

file_path = '/datasets/music_project_en.csv'
df = pd.read_csv(file_path)

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

def genre_weekday(data, day, time1, time2):
    
    genre_df = data[(data['day'] == day) & (data['time'] >= time1) & (data['time'] <= time2)]

    genre_df_count = genre_df['genre'].value_counts()

    genre_df_sorted = genre_df_count.sort_values(ascending=False)

    return genre_df_sorted[:15]

day = 'Monday'
time1 = '07:00'
time2 = '11:00'

top_genres = genre_weekday(df, day, time1, time2)

print("Top 15 popular genres on Monday from 07:00 to 11:00:")
print(top_genres)

# 1) Let the genre_df variable store the rows that meet several conditions:
#    - the value in the 'day' column is equal to the value of the day= argument
#    - the value in the 'time' column is greater than the value of the time1= argument
#    - the value in the 'time' column is smaller than the value of the time2= argument
#    Use consecutive filtering with logical indexing.

# 2) Group genre_df by the 'genre' column, take one of its columns, 
#    and use the count() method to find the number of entries for each of 
#    the represented genres; store the resulting Series to the
#    genre_df_count variable

# 3) Sort genre_df_count in descending order of frequency and store the result
#    to the genre_df_sorted variable

# 4) Return a Series object with the first 15 genre_df_sorted value - the 15 most
#    popular genres (on a given day, within a certain timeframe)

# Write your function here



    # consecutive filtering
    # Create the variable genre_df which will store only those df rows where the day is equal to day=
    
    # filter again so that genre_df will store only those rows where the time is smaller than time2=

    # filter once more so that genre_df will store only rows where the time is greater than time1=

    # group the filtered DataFrame by the column with the names of genres, take the genre column, and find the number of rows for each genre with the count() method
    
    # sort the result in descending order (so that the most popular genres come first in the Series object)
    

    # we will return the Series object storing the 15 most popular genres on a given day in a given timeframe
    

Top 15 popular genres on Monday from 07:00 to 11:00:
pop            1068
dance           781
rock            684
electronic      655
hip             393
ruspop          271
rusrap          244
alternative     240
world           229
classical       209
jazz            156
metal           154
folk            132
soundtrack      131
rnb             108
Name: genre, dtype: int64


Compare the results of the `genre_weekday()` function for Springfield and Shelbyville on Monday morning (from 7AM to 11AM) and on Friday evening (from 17:00 to 23:00):

In [31]:
spr_general = df[df['city'] == 'Springfield']
shel_general = df[df['city'] == 'Shelbyville']

def genre_weekday(data, day, time1, time2):
   
    genre_df = data[(data['day'] == day) & (data['time'] >= time1) & (data['time'] <= time2)]
    genre_df_count = genre_df['genre'].value_counts()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted[:15]


monday_morning_time1 = '07:00'
monday_morning_time2 = '11:00'

spr_monday_morning = genre_weekday(spr_general, 'Monday', monday_morning_time1, monday_morning_time2)

print("Top genres for Springfield on Monday morning:")
print(spr_monday_morning)


 # calling the function for Monday morning in Springfield (use spr_general instead of the df table)


Top genres for Springfield on Monday morning:
pop            830
dance          589
rock           511
electronic     501
hip            306
ruspop         203
world          190
rusrap         188
alternative    175
classical      167
metal          126
jazz           109
folk           107
soundtrack      97
latin           85
Name: genre, dtype: int64


In [32]:
monday_morning_time1 = '07:00'
monday_morning_time2 = '11:00'

shel_monday_morning = genre_weekday(shel_general, 'Monday', monday_morning_time1, monday_morning_time2)
# calling the function for Monday morning in Shelbyville (use shel_general instead of the df table)
shel_monday_morning

pop            238
dance          192
rock           173
electronic     154
hip             87
ruspop          68
alternative     65
rusrap          56
jazz            47
classical       42
world           39
soundtrack      34
rap             33
rnb             31
metal           28
Name: genre, dtype: int64

In [33]:
friday_evening_time1 = '17:00'
friday_evening_time2 = '23:00'

spr_friday_evening = genre_weekday(spr_general, 'Friday', friday_evening_time1, friday_evening_time2) # calling the function for Friday evening in Springfield
spr_friday_evening

pop            761
rock           546
dance          521
electronic     510
hip            276
world          220
ruspop         184
alternative    176
classical      171
rusrap         151
jazz           121
soundtrack     112
metal           92
rnb             92
folk            88
Name: genre, dtype: int64

In [34]:
friday_evening_time1 = '17:00'
friday_evening_time2 = '23:00'

shel_friday_evening = genre_weekday(shel_general, 'Friday', friday_evening_time1, friday_evening_time2) # calling the function for Friday evening in Shelbyville
shel_friday_evening

pop            279
rock           230
electronic     227
dance          221
hip            100
alternative     67
jazz            66
rusrap          66
classical       64
world           60
ruspop          49
soundtrack      40
metal           39
rap             39
latin           36
Name: genre, dtype: int64

**Conclusion**

Having compared the top 15 genres on Monday morning, we can draw the following conclusions:

1. Users from Springfield and Shelbyville listen to similar music. The top five genres are the same, only rock and electronic have switched places.

2. In Springfield, the number of missing values turned out to be so big that the value `'unknown'` came in 10th. This means that missing values make up a considerable portion of the data, which may be a basis for questioning the reliability of our conclusions.

For Friday evening, the situation is similar. Individual genres vary somewhat, but on the whole, the top 15 is similar for the two cities.

Thus, the second hypothesis has been partially proven true:
* Users listen to similar music at the beginning and end of the week.
* There is no major difference between Springfield and Shelbyville. In both cities, pop is the most popular genre.

However, the number of missing values makes this result questionable. In Springfield, there are so many that they affect our top 15. Were we not missing these values, things might look different.

[Back to Contents](#back)

### Hypothesis 3: genre preferences in Springfield and Shelbyville <a id='genre'></a>

Hypothesis: Shelbyville loves rap music. Springfield's citizens are more into pop.

Group the `spr_general` table by genre and find the number of songs played for each genre with the `count()` method. Then sort the result in descending order and store it to `spr_genres`.

In [35]:
spr_genres = spr_general['genre'].value_counts().sort_values(ascending=False) # on one line: group the spr_general table by the 'genre' column, 
# count the 'genre' values with count() in the grouping, 
# sort the resulting Series in descending order, and store it to spr_genres

Print the first 10 rows from `spr_genres`:

In [36]:
print(spr_genres.head(10))
# printing the first 10 rows of spr_genres

pop            6253
dance          4707
rock           4188
electronic     4010
hip            2156
classical      1712
world          1516
alternative    1466
ruspop         1453
rusrap         1239
Name: genre, dtype: int64


Now do the same with the data on Shelbyville.

Group the `shel_general` table by genre and find the number of songs played for each genre. Then sort the result in descending order and store it to the `shel_genres` table:


In [37]:
shel_genres = shel_general['genre'].value_counts().sort_values(ascending=False)
 # on one line: group the shel_general table by the 'genre' column, 
# count the 'genre' values in the grouping with count(), 
# sort the resulting Series in descending order and store it to shel_genres

Print the first 10 rows of `shel_genres`:

In [38]:
print(shel_genres.head(10))
 # printing the first 10 rows from shel_genres

pop            2597
dance          2054
rock           2004
electronic     1842
hip             992
alternative     700
classical       684
rusrap          604
ruspop          565
world           553
Name: genre, dtype: int64


**Conclusion**

The hypothesis has been partially proven true:
* Pop music is the most popular genre in Springfield, as expected.
* However, pop music turned out to be equally popular in Springfield and Shelbyville, and rap wasn't in the top 5 for either city.


# Findings <a id='end'></a>

We have tested the following three hypotheses:

1. User activity differs depending on the day of the week and from city to city. 
2. On Monday mornings, Springfield and Shelbyville residents listen to different genres. This is also true for Friday evenings. 
3. Springfield and Shelbyville listeners have different preferences. In both Springfield and Shelbyville, they prefer pop.

After analyzing the data, we concluded:

1. User activity in Springfield and Shelbyville depends on the day of the week, though the cities vary in different ways. 

The first hypothesis is fully accepted.

2. Musical preferences do not vary significantly over the course of the week in both Springfield and Shelbyville. We can see small differences in order on Mondays, but:
* In Springfield and Shelbyville, people listen to pop music most.

So we can't accept this hypothesis. We must also keep in mind that the result could have been different if not for the missing values.

3. It turns out that the musical preferences of users from Springfield and Shelbyville are quite similar.

The third hypothesis is rejected. If there is any difference in preferences, it cannot be seen from this data.

### Note 
In real projects, research involves statistical hypothesis testing, which is more precise and more quantitative. Also note that you cannot always draw conclusions about an entire city based on the data from just one source.

You will study hypothesis testing in the sprint on statistical data analysis.